# Import libraries

In [1]:
#Import libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None, 'display.max_columns', None) 

# Player stats

In [54]:
def get_playerstats_big5leagues(stat:str)->None:
    """
    Choose among this options : stats,keepers,keepersadv,shooting,passing,passing_types,gca,defense,possession,playingtime,misc 
    
    Parameters : stat selects one of the eleven stats available in Fbref
    
    returns : Dataframe with the whole data from Big 5 league's players for the stat choosen
    
    """
    link = 'https://fbref.com/en/comps/Big5/'+stat+'/players/Big-5-European-Leagues-Stats'
    # Creating the dataframe
    df = pd.read_html(link)
    #Choosing the table 
    df=df[0]
    #Joinning level 0 and 1 indexes
    df.columns = ['_'.join(col) for col in df.columns]
    #Dropping some rows and features useless
    df.iloc[:,0].replace('Rk',np.nan,inplace=True)
    df.dropna(subset=['Unnamed: 0_level_0_Rk'],inplace=True)
    df.drop(['Unnamed: 0_level_0_Rk', 'Unnamed: 7_level_0_Born'],axis=1,inplace=True)
    df=df.iloc[:,:-1]
    #Renaming some features
    if stat == ('stats','keepers','playingtime'):
        df.rename(columns={
        'Unnamed: 1_level_0_Player': 'Player',
        'Unnamed: 2_level_0_Nation': 'Nation', 
        'Unnamed: 3_level_0_Pos': 'Pos', 
        'Unnamed: 4_level_0_Squad':'Squad',
        'Unnamed: 5_level_0_Comp': 'Comp',
        'Unnamed: 6_level_0_Age':'Age'}, inplace=True)
    elif stat == ('passing_types'):
        df.rename(columns={
        'Unnamed: 1_level_0_Player': 'Player',
        'Unnamed: 2_level_0_Nation': 'Nation', 
        'Unnamed: 3_level_0_Pos': 'Pos', 
        'Unnamed: 4_level_0_Squad':'Squad',
        'Unnamed: 5_level_0_Comp': 'Comp',
        'Unnamed: 6_level_0_Age':'Age',
        'Unnamed: 8_level_0_90s':'90s',
        'Unnamed: 9_level_0_Att':'Att'}, inplace=True)
    else:
        df.rename(columns={
        'Unnamed: 1_level_0_Player': 'Player',
        'Unnamed: 2_level_0_Nation': 'Nation', 
        'Unnamed: 3_level_0_Pos': 'Pos', 
        'Unnamed: 4_level_0_Squad':'Squad',
        'Unnamed: 5_level_0_Comp': 'Comp',
        'Unnamed: 6_level_0_Age':'Age',
        'Unnamed: 8_level_0_90s':'90s'}, inplace=True)
    #Cleaning data in some features
    df.Age=df.Age.str.slice(0,2)
    df.Nation=df.Nation.str.slice(-4,)
    df.Comp=df.Comp.str.slice(3,)
    #Switching data type in some features
    df.iloc[:,6:]=df.iloc[:,6:].astype(float)
    #Transforming some feature names
    columnas=df.iloc[:, 6:]
    resto=df.iloc[:,:6]
    columnas.columns = [f'{stat}_{column}' for column in columnas.columns]
    df=resto.join(columnas)

    
    return df

In [55]:

#The parameter can be: stats,keepers,keepersadv,shooting,passing,passing_types,gca,defense,possession,playingtime,misc
#If you want all the stats, you must drop some features that are duplicated



#stats
stats=get_playerstats_big5leagues('stats')
stats.drop(stats.columns[[7]],axis=1,inplace=True)
#keepers
keepers=get_playerstats_big5leagues('keepers')
keepers.drop(keepers.columns[[9,10,21]],axis=1,inplace=True)
#keepersadv
keepersadv=get_playerstats_big5leagues('keepersadv')
#shooting
shooting=get_playerstats_big5leagues('shooting')
shooting.drop(shooting.columns[[6,7,17,18,19,20]],axis=1,inplace=True)
#passing
passing=get_playerstats_big5leagues('passing')
passing.drop(passing.columns[[6,22,29]],axis=1,inplace=True)
#passing_types
passing_types=get_playerstats_big5leagues('passing_types')
passing_types.drop(passing_types.columns[[6,7]],axis=1,inplace=True)
#gca
gca=get_playerstats_big5leagues('gca')
gca.drop(gca.columns[[6]],axis=1,inplace=True)
#defense
defense=get_playerstats_big5leagues('defense')
defense.drop(defense.columns[[6]],axis=1,inplace=True)
#possession
possession=get_playerstats_big5leagues('possession')
possession.drop(possession.columns[[6,22,28]],axis=1,inplace=True)
#playingtime
playingtime=get_playerstats_big5leagues('playingtime')
playingtime.drop(playingtime.columns[[6,7,10]],axis=1, inplace=True)
#misc
misc=get_playerstats_big5leagues('misc')
misc.drop(misc.columns[[6,7,8,14,15]],axis=1,inplace=True)

lista_df = [stats,keepers,keepersadv,shooting,passing,passing_types,gca,defense,possession,playingtime,misc]

In [56]:
#We merge all dfs created

a= pd.merge(playingtime, stats, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='inner')
a= pd.merge(a, keepers, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
a= pd.merge(a, keepersadv, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
a= pd.merge(a, shooting, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
a= pd.merge(a, passing, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
a= pd.merge(a, passing_types, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
a= pd.merge(a, gca, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
a= pd.merge(a, defense, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
a= pd.merge(a, possession, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
a= pd.merge(a, misc, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')

### Export the data

In [57]:
#We export the data
total_players_data_b5l=a
total_players_data_b5l.to_csv('total_players_data_b5l_.csv',encoding='UTF-8', sep=';',index=False,decimal=',')

## Player stats per 90

In [58]:
def get_90s(df:object)->None:
    """
    Parameters : df which data you want to convert in per90s
    
    returns : Dataframe with the whole data converted to per90s
    
    """
    # Specify the columns that you want to convert into per90s
    columnas = df.iloc[:, 6:]  
    resto = df.iloc[:,:6]

    # Calculate per90s for the columns selected
    n90s_df = columnas.div(df['stats_Playing Time_90s'], axis=0)

    # Optional: change the name of the columns created
    n90s_df.columns = [f'Per90s_{column}' for column in columnas.columns]

    #Join columns transformated and non-transformated
    n90s_df = resto.join(n90s_df)
    n90s_df.iloc[:,6:] = n90s_df.iloc[:,6:].round(2)
    n90s_df.head()
    
    return n90s_df

### Export the data

In [59]:
#Export data
total_players_data_b5l_per90=get_90s(a)
total_players_data_b5l_per90.to_csv('total_players_data_b5l_per90_0622.csv',encoding='UTF-8', sep=';',index=False,decimal=',')

## Player stats percentiles

In [7]:
from scipy.stats import percentileofscore
def get_percentiles(df:object)->None:
    """
    Parameters : df which data you want to convert in percentile
    
    returns : Dataframe with the whole data converted to percentile
    
    """
    # Specify the columns that you want to convert into percentiles
    columnas = df.iloc[:, 6:]  
    resto = df.iloc[:,:6]

     # Calculate percentiles for the columns selected
    percentiles_df = columnas.apply(lambda x: x.apply(lambda y: percentileofscore(columnas[x.name], y, nan_policy='omit')), axis=0)

    # Optional: change the name of the columns created
    percentiles_df.columns = [f'Percentile_{column}' for column in columnas.columns]

    #Join columns transformated and non-transformated
    percentiles_df = resto.join(percentiles_df)
    percentiles_df.iloc[:,6:] = percentiles_df.iloc[:,6:].round(2)
    percentiles_df.head()
    
    return percentiles_df

In [9]:
#percentiles

p_stats=get_percentiles(stats)
p_keepers=get_percentiles(keepers)
p_keepersadv=get_percentiles(keepersadv)
p_shooting=get_percentiles(shooting)
p_passing=get_percentiles(passing)
p_passing_types=get_percentiles(passing_types)
p_gca=get_percentiles(gca)
p_defense=get_percentiles(defense)
p_possession=get_percentiles(possession)
p_playingtime=get_percentiles(playingtime)
p_misc=get_percentiles(misc)

In [ ]:
b= pd.merge(p_playingtime, p_stats, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='inner')
b= pd.merge(b, p_keepers, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
b= pd.merge(b, p_keepersadv, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
b= pd.merge(b, p_shooting, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
b= pd.merge(b, p_passing, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
b= pd.merge(b, p_passing_types, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
b= pd.merge(b, p_gca, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
b= pd.merge(b, p_defense, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
b= pd.merge(b, p_possession, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')
b= pd.merge(b, p_misc, on=('Player','Squad','Nation', 'Pos', 'Comp', 'Age'), how='outer')

# Team stats

We follow the same path as with player stats.

In [6]:
def get_squadstats_big5leagues(stat:str)->None:
    """
    Choose among this options : stats,keepers,keepersadv,shooting,passing,passing_types,gca,defense,possession,playingtime,misc 
    
    Parameters : stat selects one of the eleven stats available in Fbref
    
    returns : Dataframe with the whole data from Big 5 league's teams for the stat choosen
    
    """
    link = 'https://fbref.com/en/comps/Big5/'+stat+'/squads/Big-5-European-Leagues-Stats'
    # Creating the dataframe
    df = pd.read_html(link)
    #Choosing the table 
    df=df[0]
    #Joinning level 0 and 1 indexes
    df.columns = ['_'.join(col) for col in df.columns]
    #Dropping some rows and features useless
    df.iloc[:,0].replace('Rk',np.nan,inplace=True)
    df.dropna(subset=['Unnamed: 0_level_0_Rk'],inplace=True)
    df.drop(['Unnamed: 0_level_0_Rk'],axis=1,inplace=True)
    #Renaming some features
    if stat == ('stats'):
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players',
        'Unnamed: 4_level_0_Age':'Age',
        'Unnamed: 5_level_0_Poss': 'Poss'}, inplace=True)
        
    elif stat == ('passing_types'):
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players',
        'Unnamed: 4_level_0_90s':'MP',
        'Unnamed: 5_level_0_Att':'Att'}, inplace=True)
        
    elif stat == ('keepers'):
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players'}, inplace=True)
        
    elif stat == ('possession'):
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players',
        'Unnamed: 4_level_0_Poss':'Poss',
        'Unnamed: 5_level_0_90s':'MP'}, inplace=True)
        
    elif stat == ('playingtime'):
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players',
        'Unnamed: 4_level_0_Age':'Age'}, inplace=True)
        
    else:
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players',
        'Unnamed: 4_level_0_90s':'MP'}, inplace=True)
        
    #Cleaning data in some features
    df.Comp=df.Comp.str.slice(3,)
    #Transforming some feature names
    columnas=df.iloc[:, 2:]
    resto=df.iloc[:,:2]
    columnas.columns = [f'{stat}_{column}' for column in columnas.columns]
    df=resto.join(columnas)

    
    return df

In [7]:
#stats,keepers,keepersadv,shooting,passing,passing_types,gca,defense,possession,playingtime,misc

stats=get_squadstats_big5leagues('stats')
keepers=get_squadstats_big5leagues('keepers')
keepersadv=get_squadstats_big5leagues('keepersadv')
shooting=get_squadstats_big5leagues('shooting')
passing=get_squadstats_big5leagues('passing')
passing_types=get_squadstats_big5leagues('passing_types')
gca=get_squadstats_big5leagues('gca')
defense=get_squadstats_big5leagues('defense')
possession=get_squadstats_big5leagues('possession')
playingtime=get_squadstats_big5leagues('playingtime')
misc=get_squadstats_big5leagues('misc')

lista_df = [stats,keepers,keepersadv,shooting,passing,passing_types,gca,defense,possession,playingtime,misc]

In [31]:
stats.head()

,Squad,Comp,stats_N_players,stats_Age,stats_Poss,stats_Playing Time_MP,stats_Playing Time_Starts,stats_Playing Time_Min,stats_Playing Time_90s,stats_Performance_Gls,stats_Performance_Ast,stats_Performance_G+A,stats_Performance_G-PK,stats_Performance_PK,stats_Performance_PKatt,stats_Performance_CrdY,stats_Performance_CrdR,stats_Expected_xG,stats_Expected_npxG,stats_Expected_xAG,stats_Expected_npxG+xAG,stats_Progression_PrgC,stats_Progression_PrgP,stats_Per 90 Minutes_Gls,stats_Per 90 Minutes_Ast,stats_Per 90 Minutes_G+A,stats_Per 90 Minutes_G-PK,stats_Per 90 Minutes_G+A-PK,stats_Per 90 Minutes_xG,stats_Per 90 Minutes_xAG,stats_Per 90 Minutes_xG+xAG,stats_Per 90 Minutes_npxG,stats_Per 90 Minutes_npxG+xAG
0,Ajaccio,Ligue 1,36,30.0,44.2,36,396,3240,36.0,21,12,33,15,6,9,83,10,35.7,29.1,22.4,51.5,475,1143,0.58,0.33,0.92,0.42,0.75,0.99,0.62,1.62,0.81,1.43
1,Almería,La Liga,29,27.2,45.3,36,396,3240,36.0,46,31,77,45,1,2,94,4,42.0,40.5,30.2,70.7,538,1025,1.28,0.86,2.14,1.25,2.11,1.17,0.84,2.01,1.12,1.96
2,Angers,Ligue 1,33,26.6,46.0,36,396,3240,36.0,29,17,46,25,4,4,68,5,39.4,36.3,26.9,63.2,602,1302,0.81,0.47,1.28,0.69,1.17,1.10,0.75,1.84,1.01,1.76
3,Arsenal,Premier League,26,25.5,59.5,37,407,3330,37.0,79,60,139,76,3,4,51,0,69.1,66.3,51.3,117.6,804,2004,2.14,1.62,3.76,2.05,3.68,1.87,1.39,3.25,1.79,3.18
4,Aston Villa,Premier League,26,27.8,49.5,37,407,3330,37.0,47,33,80,44,3,4,76,1,47.4,44.3,36.2,80.5,617,1215,1.27,0.89,2.16,1.19,2.08,1.28,0.98,2.26,1.20,2.18


In [8]:
a= pd.merge(playingtime, stats, on=('Squad','Comp'), how='inner')
a= pd.merge(a, keepers, on=('Squad','Comp'), how='outer')
a= pd.merge(a, keepersadv, on=('Squad','Comp'), how='outer')
a= pd.merge(a, shooting, on=('Squad','Comp'), how='outer')
a= pd.merge(a, passing, on=('Squad','Comp'), how='outer')
a= pd.merge(a, passing_types, on=('Squad','Comp'), how='outer')
a= pd.merge(a, gca, on=('Squad','Comp'), how='outer')
a= pd.merge(a, defense, on=('Squad','Comp'), how='outer')
a= pd.merge(a, possession, on=('Squad','Comp'), how='outer')
a= pd.merge(a, misc, on=('Squad','Comp'), how='outer')

a.head()

,Squad,Comp,playingtime_N_players,playingtime_Age,playingtime_Playing Time_MP,playingtime_Playing Time_Min,playingtime_Playing Time_Mn/MP,playingtime_Playing Time_Min%,playingtime_Playing Time_90s,playingtime_Starts_Starts,playingtime_Starts_Mn/Start,playingtime_Starts_Compl,playingtime_Subs_Subs,playingtime_Subs_Mn/Sub,playingtime_Subs_unSub,playingtime_Team Success_PPM,playingtime_Team Success_onG,playingtime_Team Success_onGA,playingtime_Team Success_+/-,playingtime_Team Success_+/-90,playingtime_Team Success (xG)_onxG,playingtime_Team Success (xG)_onxGA,playingtime_Team Success (xG)_xG+/-,playingtime_Team Success (xG)_xG+/-90,stats_N_players,stats_Age,stats_Poss,stats_Playing Time_MP,stats_Playing Time_Starts,stats_Playing Time_Min,stats_Playing Time_90s,stats_Performance_Gls,stats_Performance_Ast,stats_Performance_G+A,stats_Performance_G-PK,stats_Performance_PK,stats_Performance_PKatt,stats_Performance_CrdY,stats_Performance_CrdR,stats_Expected_xG,stats_Expected_npxG,stats_Expected_xAG,stats_Expected_npxG+xAG,stats_Progression_PrgC,stats_Progression_PrgP,stats_Per 90 Minutes_Gls,stats_Per 90 Minutes_Ast,stats_Per 90 Minutes_G+A,stats_Per 90 Minutes_G-PK,stats_Per 90 Minutes_G+A-PK,stats_Per 90 Minutes_xG,stats_Per 90 Minutes_xAG,stats_Per 90 Minutes_xG+xAG,stats_Per 90 Minutes_npxG,stats_Per 90 Minutes_npxG+xAG,keepers_N_players,keepers_Playing Time_MP,keepers_Playing Time_Starts,keepers_Playing Time_Min,keepers_Unnamed: 7_level_0_90s,keepers_Performance_GA,keepers_Performance_GA90,keepers_Performance_SoTA,keepers_Performance_Saves,keepers_Performance_Save%,keepers_Performance_W,keepers_Performance_D,keepers_Performance_L,keepers_Performance_CS,keepers_Performance_CS%,keepers_Penalty Kicks_PKatt,keepers_Penalty Kicks_PKA,keepers_Penalty Kicks_PKsv,keepers_Penalty Kicks_PKm,keepers_Penalty Kicks_Save%,keepersadv_N_players,keepersadv_MP,keepersadv_Goals_GA,keepersadv_Goals_PKA,keepersadv_Goals_FK,keepersadv_Goals_CK,keepersadv_Goals_OG,keepersadv_Expected_PSxG,keepersadv_Expected_PSxG/SoT,keepersadv_Expected_PSxG+/-,keepersadv_Expected_/90,keepersadv_Launched_Cmp,keepersadv_Launched_Att,keepersadv_Launched_Cmp%,keepersadv_Passes_Att,keepersadv_Passes_Thr,keepersadv_Passes_Launch%,keepersadv_Passes_AvgLen,keepersadv_Goal Kicks_Att,keepersadv_Goal Kicks_Launch%,keepersadv_Goal Kicks_AvgLen,keepersadv_Crosses_Opp,keepersadv_Crosses_Stp,keepersadv_Crosses_Stp%,keepersadv_Sweeper_#OPA,keepersadv_Sweeper_#OPA/90,keepersadv_Sweeper_AvgDist,shooting_N_players,shooting_MP,shooting_Standard_Gls,shooting_Standard_Sh,shooting_Standard_SoT,shooting_Standard_SoT%,shooting_Standard_Sh/90,shooting_Standard_SoT/90,shooting_Standard_G/Sh,shooting_Standard_G/SoT,shooting_Standard_Dist,shooting_Standard_FK,shooting_Standard_PK,shooting_Standard_PKatt,shooting_Expected_xG,shooting_Expected_npxG,shooting_Expected_npxG/Sh,shooting_Expected_G-xG,shooting_Expected_np:G-xG,passing_N_players,passing_MP,passing_Total_Cmp,passing_Total_Att,passing_Total_Cmp%,passing_Total_TotDist,passing_Total_PrgDist,passing_Short_Cmp,passing_Short_Att,passing_Short_Cmp%,passing_Medium_Cmp,passing_Medium_Att,passing_Medium_Cmp%,passing_Long_Cmp,passing_Long_Att,passing_Long_Cmp%,passing_Unnamed: 19_level_0_Ast,passing_Unnamed: 20_level_0_xAG,passing_Unnamed: 21_level_0_xA,passing_Unnamed: 22_level_0_A-xAG,passing_Unnamed: 23_level_0_KP,passing_Unnamed: 24_level_0_1/3,passing_Unnamed: 25_level_0_PPA,passing_Unnamed: 26_level_0_CrsPA,passing_Unnamed: 27_level_0_PrgP,passing_types_N_players,passing_types_MP,passing_types_Att,passing_types_Pass Types_Live,passing_types_Pass Types_Dead,passing_types_Pass Types_FK,passing_types_Pass Types_TB,passing_types_Pass Types_Sw,passing_types_Pass Types_Crs,passing_types_Pass Types_TI,passing_types_Pass Types_CK,passing_types_Corner Kicks_In,passing_types_Corner Kicks_Out,passing_types_Corner Kicks_Str,passing_types_Outcomes_Cmp,passing_types_Outcomes_Off,passing_types_Outcomes_Blocks,gca_N_players,gca_MP,gca_SCA_SCA,gca_SCA_SCA90,gc

### Export the data

In [9]:
total_squaddata_b5l=a
total_squaddata_b5l.to_csv(r'total_squad_data_b5l.csv',encoding='UTF-8', sep=';',index=False,decimal=',')

# VS Team stats

In [10]:
def get_vssquadstats_big5leagues(stat:str)->None:
    """
    Choose among this options : stats,keepers,keepersadv,shooting,passing,passing_types,gca,defense,possession,playingtime,misc 
    
    Parameters : stat selects one of the eleven stats available in Fbref
    
    returns : Dataframe with the whole data from Big 5 league's teams for the stat choosen
    
    """
    link = 'https://fbref.com/en/comps/Big5/'+stat+'/squads/Big-5-European-Leagues-Stats'
    # Creating the dataframe
    df = pd.read_html(link)
    #Choosing the table 
    df=df[1]
    #Joinning level 0 and 1 indexes
    df.columns = ['_'.join(col) for col in df.columns]
    #Dropping some rows and features useless
    df.iloc[:,0].replace('Rk',np.nan,inplace=True)
    df.dropna(subset=['Unnamed: 0_level_0_Rk'],inplace=True)
    df.drop(['Unnamed: 0_level_0_Rk'],axis=1,inplace=True)
    #Renaming some features
    if stat == ('stats'):
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players',
        'Unnamed: 4_level_0_Age':'Age',
        'Unnamed: 5_level_0_Poss': 'Poss'}, inplace=True)
        
    elif stat == ('passing_types'):
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players',
        'Unnamed: 4_level_0_90s':'MP',
        'Unnamed: 5_level_0_Att':'Att'}, inplace=True)
        
    elif stat == ('keepers'):
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players'}, inplace=True)
        
    elif stat == ('possession'):
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players',
        'Unnamed: 4_level_0_Poss':'Poss',
        'Unnamed: 5_level_0_90s':'MP'}, inplace=True)
        
    elif stat == ('playingtime'):
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players',
        'Unnamed: 4_level_0_Age':'Age'}, inplace=True)
        
    else:
        df.rename(columns={
        'Unnamed: 1_level_0_Squad':'Squad',
        'Unnamed: 2_level_0_Comp': 'Comp',
        'Unnamed: 3_level_0_# Pl': 'N_players',
        'Unnamed: 4_level_0_90s':'MP'}, inplace=True)
        
    #Cleaning data in some features
    df.Comp=df.Comp.str.slice(3,)
    #Transforming some feature names
    columnas=df.iloc[:, 2:]
    resto=df.iloc[:,:2]
    columnas.columns = [f'{stat}_{column}' for column in columnas.columns]
    df=resto.join(columnas)

    
    return df

In [11]:
#stats,keepers,keepersadv,shooting,passing,passing_types,gca,defense,possession,playingtime,misc

stats=get_vssquadstats_big5leagues('stats')
keepers=get_vssquadstats_big5leagues('keepers')
keepersadv=get_vssquadstats_big5leagues('keepersadv')
shooting=get_vssquadstats_big5leagues('shooting')
passing=get_vssquadstats_big5leagues('passing')
passing_types=get_vssquadstats_big5leagues('passing_types')
gca=get_vssquadstats_big5leagues('gca')
defense=get_vssquadstats_big5leagues('defense')
possession=get_vssquadstats_big5leagues('possession')
playingtime=get_vssquadstats_big5leagues('playingtime')
misc=get_vssquadstats_big5leagues('misc')

lista_df = [stats,keepers,keepersadv,shooting,passing,passing_types,gca,defense,possession,playingtime,misc]

In [12]:
a= pd.merge(playingtime, stats, on=('Squad','Comp'), how='inner')
a= pd.merge(a, keepers, on=('Squad','Comp'), how='outer')
a= pd.merge(a, keepersadv, on=('Squad','Comp'), how='outer')
a= pd.merge(a, shooting, on=('Squad','Comp'), how='outer')
a= pd.merge(a, passing, on=('Squad','Comp'), how='outer')
a= pd.merge(a, passing_types, on=('Squad','Comp'), how='outer')
a= pd.merge(a, gca, on=('Squad','Comp'), how='outer')
a= pd.merge(a, defense, on=('Squad','Comp'), how='outer')
a= pd.merge(a, possession, on=('Squad','Comp'), how='outer')
a= pd.merge(a, misc, on=('Squad','Comp'), how='outer')

a.head()

,Squad,Comp,playingtime_N_players,playingtime_Age,playingtime_Playing Time_MP,playingtime_Playing Time_Min,playingtime_Playing Time_Mn/MP,playingtime_Playing Time_Min%,playingtime_Playing Time_90s,playingtime_Starts_Starts,playingtime_Starts_Mn/Start,playingtime_Starts_Compl,playingtime_Subs_Subs,playingtime_Subs_Mn/Sub,playingtime_Subs_unSub,playingtime_Team Success_PPM,playingtime_Team Success_onG,playingtime_Team Success_onGA,playingtime_Team Success_+/-,playingtime_Team Success_+/-90,playingtime_Team Success (xG)_onxG,playingtime_Team Success (xG)_onxGA,playingtime_Team Success (xG)_xG+/-,playingtime_Team Success (xG)_xG+/-90,stats_N_players,stats_Age,stats_Poss,stats_Playing Time_MP,stats_Playing Time_Starts,stats_Playing Time_Min,stats_Playing Time_90s,stats_Performance_Gls,stats_Performance_Ast,stats_Performance_G+A,stats_Performance_G-PK,stats_Performance_PK,stats_Performance_PKatt,stats_Performance_CrdY,stats_Performance_CrdR,stats_Expected_xG,stats_Expected_npxG,stats_Expected_xAG,stats_Expected_npxG+xAG,stats_Progression_PrgC,stats_Progression_PrgP,stats_Per 90 Minutes_Gls,stats_Per 90 Minutes_Ast,stats_Per 90 Minutes_G+A,stats_Per 90 Minutes_G-PK,stats_Per 90 Minutes_G+A-PK,stats_Per 90 Minutes_xG,stats_Per 90 Minutes_xAG,stats_Per 90 Minutes_xG+xAG,stats_Per 90 Minutes_npxG,stats_Per 90 Minutes_npxG+xAG,keepers_N_players,keepers_Playing Time_MP,keepers_Playing Time_Starts,keepers_Playing Time_Min,keepers_Unnamed: 7_level_0_90s,keepers_Performance_GA,keepers_Performance_GA90,keepers_Performance_SoTA,keepers_Performance_Saves,keepers_Performance_Save%,keepers_Performance_W,keepers_Performance_D,keepers_Performance_L,keepers_Performance_CS,keepers_Performance_CS%,keepers_Penalty Kicks_PKatt,keepers_Penalty Kicks_PKA,keepers_Penalty Kicks_PKsv,keepers_Penalty Kicks_PKm,keepers_Penalty Kicks_Save%,keepersadv_N_players,keepersadv_MP,keepersadv_Goals_GA,keepersadv_Goals_PKA,keepersadv_Goals_FK,keepersadv_Goals_CK,keepersadv_Goals_OG,keepersadv_Expected_PSxG,keepersadv_Expected_PSxG/SoT,keepersadv_Expected_PSxG+/-,keepersadv_Expected_/90,keepersadv_Launched_Cmp,keepersadv_Launched_Att,keepersadv_Launched_Cmp%,keepersadv_Passes_Att,keepersadv_Passes_Thr,keepersadv_Passes_Launch%,keepersadv_Passes_AvgLen,keepersadv_Goal Kicks_Att,keepersadv_Goal Kicks_Launch%,keepersadv_Goal Kicks_AvgLen,keepersadv_Crosses_Opp,keepersadv_Crosses_Stp,keepersadv_Crosses_Stp%,keepersadv_Sweeper_#OPA,keepersadv_Sweeper_#OPA/90,keepersadv_Sweeper_AvgDist,shooting_N_players,shooting_MP,shooting_Standard_Gls,shooting_Standard_Sh,shooting_Standard_SoT,shooting_Standard_SoT%,shooting_Standard_Sh/90,shooting_Standard_SoT/90,shooting_Standard_G/Sh,shooting_Standard_G/SoT,shooting_Standard_Dist,shooting_Standard_FK,shooting_Standard_PK,shooting_Standard_PKatt,shooting_Expected_xG,shooting_Expected_npxG,shooting_Expected_npxG/Sh,shooting_Expected_G-xG,shooting_Expected_np:G-xG,passing_N_players,passing_MP,passing_Total_Cmp,passing_Total_Att,passing_Total_Cmp%,passing_Total_TotDist,passing_Total_PrgDist,passing_Short_Cmp,passing_Short_Att,passing_Short_Cmp%,passing_Medium_Cmp,passing_Medium_Att,passing_Medium_Cmp%,passing_Long_Cmp,passing_Long_Att,passing_Long_Cmp%,passing_Unnamed: 19_level_0_Ast,passing_Unnamed: 20_level_0_xAG,passing_Unnamed: 21_level_0_xA,passing_Unnamed: 22_level_0_A-xAG,passing_Unnamed: 23_level_0_KP,passing_Unnamed: 24_level_0_1/3,passing_Unnamed: 25_level_0_PPA,passing_Unnamed: 26_level_0_CrsPA,passing_Unnamed: 27_level_0_PrgP,passing_types_N_players,passing_types_MP,passing_types_Att,passing_types_Pass Types_Live,passing_types_Pass Types_Dead,passing_types_Pass Types_FK,passing_types_Pass Types_TB,passing_types_Pass Types_Sw,passing_types_Pass Types_Crs,passing_types_Pass Types_TI,passing_types_Pass Types_CK,passing_types_Corner Kicks_In,passing_types_Corner Kicks_Out,passing_types_Corner Kicks_Str,passing_types_Outcomes_Cmp,passing_types_Outcomes_Off,passing_types_Outcomes_Blocks,gca_N_players,gca_MP,gca_SCA_SCA,gca_SCA_SCA90,gc

### Export the data

In [13]:
total_vssquaddata_b5l=a
total_vssquaddata_b5l.to_csv(r'total_vssquad_data_b5l.csv',encoding='UTF-8', sep=';',index=False,decimal=',')